In [18]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import csv
driver=webdriver.Chrome()
driver.maximize_window()
driver.get('https://hbcglobalartcollection.com/artists/')
time.sleep(5)
html = driver.page_source
soup = BeautifulSoup(html, 'html.parser')

In [ ]:
artist_links = []

for link in soup.find_all('a',href=True):
   
    if 'artist' in link['href']:
        artist_links.append(link)
unique_keys = set()
collection_data = []
for artist in artist_links:
    artist_name = artist.text.strip()
    artist_url = artist['href']
    print(f'\nArtist Name: {artist_name}\nArtist URL: {artist_url}')
    driver.get(artist_url)
    time.sleep(3) 
    artist_html = driver.page_source
    artist_soup = BeautifulSoup(artist_html, 'html.parser')
    artist_name=artist_soup.find('div',class_="inner-wrap").find('h1').text.strip()
    print(artist_name)
    collection_elements = artist_soup.find_all('article', class_='result')
    
    for collection in collection_elements:
        
        collection_link = collection.find('a')
        if collection_link:
        
         
            collection_url = collection_link.get('href')
            
            driver.get(collection_url)
            time.sleep(10)
            collection_html = driver.page_source
            collection_soup = BeautifulSoup(collection_html, 'html.parser')
            image_url = 'No image found'
            div_tag = collection_soup.find('div', class_='col span_9')
            first_slide = collection_soup.find('div', class_='swiper-slide')
            if first_slide:
                image_bg = first_slide.find('div', class_='image-bg')
                style = image_bg.get('style', '')
                
                start = style.find('url(') + 4
                end = style.find(')', start)
                first_image_url = style[start:end]
                print("image URL:", first_image_url)
            elif div_tag:
                image_tag = div_tag.find('img')
                if image_tag and image_tag.get('src'):
                    image_url = image_tag['src']
                    print(image_url)
            
            details = {'Artist Name': artist_name, 'Collection URL': collection_url,'collection_img_url':image_url}
        
        
            collection_details = collection_soup.find_all('div', class_='project-meta')
            for detail in collection_details:
                key = detail.find('h4').text.strip().lower().replace(" ", "_")  
                value = detail.find('p').text.strip()  
                
                  
                unique_keys.add(key) 
                details[key] = value
            collection_data.append(details)        

        else:
            print("No collection URL found for this item.")
            
headers = ['Artist Name', 'Collection URL'] + sorted(unique_keys)

with open('art_collection_data.csv', mode='w', newline='', encoding='utf-8') as file:
    writer = csv.DictWriter(file, fieldnames=headers)
    writer.writeheader()

    for data in collection_data:
        writer.writerow(data)    


Artist Name: Aaron Siskind
Artist URL: https://hbcglobalartcollection.com/artist/aaron-siskind/
Aaron Siskind
First image URL (from style): https://hbcglobalartcollection.com/wp-content/uploads/2017/08/1-3.jpg

Artist Name: Adam Lampton
Artist URL: https://hbcglobalartcollection.com/artist/adam-lampton/
Adam Lampton
https://hbcglobalartcollection.com/wp-content/uploads/2016/11/BAF00866_AdamLampton_Macau-Jockey-Club.jpg

Artist Name: Adam Magyar
Artist URL: https://hbcglobalartcollection.com/artist/adam-magyar/
Adam Magyar
https://hbcglobalartcollection.com/wp-content/uploads/2017/08/AdamMagyar_Squares2.jpg
https://hbcglobalartcollection.com/wp-content/uploads/2017/08/AdamMagyar_Squares1.jpg

Artist Name: Adriana Lestido
Artist URL: https://hbcglobalartcollection.com/artist/adriana-lestido/
Adriana Lestido
https://hbcglobalartcollection.com/wp-content/uploads/2017/08/AdrianaLestido.jpg
https://hbcglobalartcollection.com/wp-content/uploads/2017/03/photo1-13.jpg
https://hbcglobalartcolle

In [ ]:
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import TimeoutException
from bs4 import BeautifulSoup
import time
import csv
driver=webdriver.Chrome()
driver.maximize_window()


def get_artist_links():
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')
    artist_links = []
    for link in soup.find_all('a', href=True):
        if 'artist' in link['href']:
            artist_links.append(link)
    return artist_links
def get_collection_links(artist_url):
    
    driver.get(artist_url)
    time.sleep(3)
    artist_html = driver.page_source
    artist_soup = BeautifulSoup(artist_html, 'html.parser')

    collection_urls = []
    collection_elements = artist_soup.find_all('article', class_='result')

    for collection in collection_elements:
        collection_link = collection.find('a')
        if collection_link:
            collection_url = collection_link.get('href')
            collection_urls.append(collection_url)
        else:
            print("No collection URL found for this item.")
    
    return collection_urls
def extract_collection_details(collection_url):
    driver.get(collection_url)
    time.sleep(5)
    collection_html = driver.page_source
    collection_soup = BeautifulSoup(collection_html, 'html.parser')

    details = {}
    image_url = 'No image found'
    div_tag = collection_soup.find('div', class_='col span_9')
    first_slide = collection_soup.find('div', class_='swiper-slide')

    
    if first_slide:
        image_bg = first_slide.find('div', class_='image-bg')
        style = image_bg.get('style', '')
        
        start = style.find('url(') + 4
        end = style.find(')', start)
        first_image_url = style[start:end]
        print("First image URL (from style):", first_image_url)
    elif div_tag:
        image_tag = div_tag.find('img')
        if image_tag and image_tag.get('src'):
            image_url = image_tag['src']
            print(image_url)
        

       
    details['collection_img_url'] = image_url
    project_detail = collection_soup.find_all('div', class_='project-meta')
    for detail in project_detail:
        title_element = detail.find('h4') 
        value_element = detail.find('p')  

        key = title_element.text.strip().lower().replace(" ", "_")  
        value = value_element.text.strip()  
        details[key] = value 

    
    return details

def main():
 
    driver.get('https://hbcglobalartcollection.com/artists/')
    time.sleep(3)
    html = driver.page_source
    soup = BeautifulSoup(html, 'html.parser')

    unique_keys = set()
    artist_links = get_artist_links()

    for artist in artist_links:
        artist_name = artist.text.strip()
        artist_url = artist['href']
        print(f'\nArtist Name: {artist_name}\nArtist URL: {artist_url}')

        collection_urls = get_collection_links(artist_url)
        
        
        for collection_url in collection_urls:
            
            collection_details = extract_collection_details(collection_url)
            unique_keys.update(collection_details.keys())
           
    headers = ['Artist Name', 'Artist URL', 'Collection URL','collection_img_url'] + sorted(unique_keys)
    with open('art_collection_data.csv', mode='w', newline='', encoding='utf-8') as file:
        writer = csv.writer(file)
        writer.writerow(headers)      
        for artist in artist_links:
            artist_name = artist.text.strip()
            artist_url = artist['href']
            collection_urls = get_collection_links(artist_url)
            for collection_url in collection_urls:
                collection_details = extract_collection_details(collection_url)
                row = [artist_name, artist_url, collection_url,image_url]
                row += [collection_details.get(key, '') for key in sorted(unique_keys)]
                writer.writerow(row)    

   

if __name__ == '__main__':
    main()